In [1]:
import pandas as pd
import numpy as np
import os
import sys


In [2]:
%pwd

'd:\\VS code files\\store_item_demand_forecasting_challenge\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\VS code files\\store_item_demand_forecasting_challenge'

In [5]:
os.listdir()

['.git',
 '.gitignore',
 'app.py',
 'config',
 'Dockerfile',
 'logs',
 'params.yaml',
 'README.md',
 'requirements.txt',
 'research',
 'schema.yaml',
 'setup.py',
 'src',
 'template.py']

In [22]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [23]:
from src.store_demand_forecasting.constants import *
from src.store_demand_forecasting.utils.common import read_yaml, create_directories

In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [28]:
import os
import urllib.request as request
import zipfile
from src.store_demand_forecasting.logger import logging
from src.store_demand_forecasting.utils.common import get_size

In [54]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logging.info(f"{filename} download! with following info: \n{headers}")
        else:
            logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(unzip_path, 'w') as myzip:
            myzip.write('eggs.csv')
        # with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
        #     zip_ref.extractall(unzip_path)

In [55]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-21 11:47:26,670: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-21 11:47:26,673: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-21 11:47:26,674: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-21 11:47:26,675: INFO: common: created directory at: artifacts]
[2024-04-21 11:47:26,676: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-21 11:47:26,676: INFO: 2894339320: File already exists of size: ~ 148 KB]


PermissionError: [Errno 13] Permission denied: 'artifacts/data_ingestion'

In [12]:
# import requests
# url = "https://github.com/aakashsyadav1999/Store-Item-Demand-Forecasting-Challenge/blob/main/research/demand-forecasting-kernels-only.zip"
# response = requests.get(url, stream=True)

In [13]:
#with open("gdp_by_country.zip", mode="wb") as file:
    # file.write(response.content)